<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_nlp/blob/main/%D0%93%D0%BB%D0%B0%D0%B4%D1%8B%D1%88%D0%B5%D0%B2%D0%92%D0%92_NLP_HW_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 5 Гладышев ВВ

In [45]:
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline

from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

In [33]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score

from sklearn.preprocessing import LabelEncoder

In [3]:
! pip install pyconll

In [4]:
import pyconll

In [5]:
!wget -O ./ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2024-02-19 12:53:12--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736599 (39M) [text/plain]
Saving to: ‘./ru_syntagrus-ud-train.conllu’

./ru_syntagrus-ud-t 100%[===================>]  38.85M   166MB/s    in 0.2s    

2024-02-19 12:53:13 (166 MB/s) - ‘./ru_syntagrus-ud-train.conllu’ saved [40736599/40736599]

--2024-02-19 12:53:13--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199

In [6]:
full_train = pyconll.load_from_file('/content/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('/content/ru_syntagrus-ud-dev.conllu')

In [7]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])

fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])

fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [8]:
len(fdata_train), len(fdata_test), len(fdata_sent_test), fdata_train[0]

(24516, 8906, 8906, [('Анкета', 'NOUN'), ('.', 'PUNCT')])

In [9]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(str(token.form)) for sent in full_train for token in sent)
print('Наибольшая длина предложения:', MAX_SENT_LEN)
print('Наибольшая длина токена:', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения: 194
Наибольшая длина токена: 31


# Задание 1

Написать теггер на данных с русским языком
* проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
* написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
* сравнить все реализованные методы, сделать выводы  

In [10]:
unigram_tagger = UnigramTagger(fdata_train)
acc_unigram_tagger = unigram_tagger.evaluate(fdata_test)
display(unigram_tagger.tag(fdata_sent_test[100]), acc_unigram_tagger)

<ipython-input-10-108c5a88a30c>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_unigram_tagger = unigram_tagger.evaluate(fdata_test)


[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', None),
 ('XVI', 'NUM'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.823732013802982

In [16]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
acc_bigram_tagger = bigram_tagger.evaluate(fdata_test)
display(acc_bigram_tagger)

<ipython-input-16-d9c76002e4ba>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_bigram_tagger = bigram_tagger.evaluate(fdata_test)


0.8292792499511688

In [17]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
acc_trigram_tagger = trigram_tagger.evaluate(fdata_test)
display(acc_trigram_tagger)

<ipython-input-17-ecb048d4a987>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_trigram_tagger = trigram_tagger.evaluate(fdata_test)


0.8291425222996289

In [18]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN')
tag = backoff_tagger(fdata_train,
                     [UnigramTagger, BigramTagger, TrigramTagger],
                     backoff = backoff)

acc_uni_bi_trigramtagger = tag.evaluate(fdata_test)
display(acc_uni_bi_trigramtagger)

<ipython-input-18-8a307cb991ee>:12: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_uni_bi_trigramtagger = tag.evaluate(fdata_test)


0.827905462595221

In [19]:
tag = backoff_tagger(fdata_train,
                     [UnigramTagger, BigramTagger],
                     backoff = backoff)

acc_uni_bigramtagger = tag.evaluate(fdata_test)
display(acc_uni_bigramtagger)

<ipython-input-19-1275b9ad503b>:5: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_uni_bigramtagger = tag.evaluate(fdata_test)


0.8275343446838986

In [27]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])

test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append('' if tok[0] is None else tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [28]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=50)
X_train = hvectorizer.fit_transform(train_tok)
type(X_train), X_train.shape

(scipy.sparse._csr.csr_matrix, (426182, 50))

In [29]:
train_tok[0], test_tok[0]

('Анкета', 'Алгоритм')

In [30]:
X_test = hvectorizer.transform(test_tok)

In [31]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [32]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=50)
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)
lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
acc_HashingVectorizer = accuracy_score(test_enc_labels, pred)
acc_HashingVectorizer

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5433687089003191

In [36]:
print(classification_report(test_enc_labels, pred, target_names=le.classes_))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         ADJ       0.32      0.26      0.29     15103
         ADP       0.52      0.79      0.63     13717
         ADV       0.29      0.04      0.07      7783
         AUX       0.00      0.00      0.00      1390
       CCONJ       0.82      0.70      0.76      5672
         DET       0.00      0.00      0.00      4265
        INTJ       0.00      0.00      0.00        24
        NOUN       0.41      0.73      0.53     36238
      NO_TAG       0.00      0.00      0.00       265
         NUM       0.00      0.00      0.00      1734
        PART       0.84      0.40      0.54      5125
        PRON       0.33      0.16      0.21      7444
       PROPN       0.00      0.00      0.00      5473
       PUNCT       0.91      0.94      0.93     29186
       SCONJ       0.67      0.68      0.67      2865
         SYM       0.00      0.00      0.00        62
        VERB       0.46      0.31      0.37     17110
           X       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=64)
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)
lr = LogisticRegression(random_state=0, max_iter=1000)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
acc_HashingVectorizer_w = accuracy_score(test_enc_labels, pred)
acc_HashingVectorizer_w

0.26066150139983074

In [44]:
for i in range(6, 9):
    print(f'HashingVectorizer features: {2**i}')
    vec = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=2**i)
    X_train = vec.fit_transform(train_tok)
    X_test = vec.transform(test_tok)
    clf = LogisticRegression(random_state=42, max_iter=500)
    clf.fit(X_train, train_enc_labels)
    pred = clf.predict(X_test)
    print(classification_report(test_enc_labels, pred, target_names=le.classes_))
    print()

HashingVectorizer features: 64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels wit

              precision    recall  f1-score   support

         ADJ       0.51      0.41      0.45     15103
         ADP       0.73      0.88      0.80     13717
         ADV       0.40      0.24      0.30      7783
         AUX       0.81      0.95      0.88      1390
       CCONJ       0.82      0.98      0.89      5672
         DET       0.54      0.35      0.43      4265
        INTJ       0.00      0.00      0.00        24
        NOUN       0.53      0.70      0.60     36238
      NO_TAG       1.00      0.77      0.87       265
         NUM       0.44      0.34      0.38      1734
        PART       0.69      0.62      0.66      5125
        PRON       0.65      0.69      0.67      7444
       PROPN       0.30      0.01      0.02      5473
       PUNCT       0.99      1.00      1.00     29186
       SCONJ       0.67      0.86      0.75      2865
         SYM       0.93      0.69      0.80        62
        VERB       0.56      0.44      0.49     17110
           X       0.24    

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels wit

              precision    recall  f1-score   support

         ADJ       0.62      0.57      0.59     15103
         ADP       0.84      0.97      0.90     13717
         ADV       0.56      0.40      0.47      7783
         AUX       0.81      0.96      0.88      1390
       CCONJ       0.85      0.99      0.91      5672
         DET       0.63      0.53      0.58      4265
        INTJ       0.00      0.00      0.00        24
        NOUN       0.63      0.75      0.69     36238
      NO_TAG       1.00      0.77      0.87       265
         NUM       0.69      0.68      0.68      1734
        PART       0.81      0.69      0.74      5125
        PRON       0.71      0.81      0.76      7444
       PROPN       0.46      0.13      0.20      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.76      0.90      0.82      2865
         SYM       0.90      0.69      0.78        62
        VERB       0.68      0.60      0.63     17110
           X       0.33    

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         ADJ       0.72      0.69      0.71     15103
         ADP       0.93      0.98      0.95     13717
         ADV       0.67      0.59      0.63      7783
         AUX       0.81      0.96      0.88      1390
       CCONJ       0.87      1.00      0.93      5672
         DET       0.83      0.66      0.74      4265
        INTJ       0.00      0.00      0.00        24
        NOUN       0.73      0.82      0.77     36238
      NO_TAG       1.00      0.77      0.87       265
         NUM       0.74      0.77      0.75      1734
        PART       0.93      0.73      0.82      5125
        PRON       0.79      0.89      0.84      7444
       PROPN       0.55      0.23      0.32      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.80      0.90      0.84      2865
         SYM       0.96      0.85      0.91        62
        VERB       0.75      0.71      0.73     17110
           X       0.40    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1024)
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)
lr = LogisticRegression(random_state=0, max_iter=2000)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
acc_HashingVectorizer_1024 = accuracy_score(test_enc_labels, pred)
acc_HashingVectorizer_1024

0.8856956833127156

In [49]:
result = pd.DataFrame({ "method": ["UnigramTagger", "BigramTagger", "TrigramTagger", "Uni BigramTagger", "Uni Bi TrigramTagger", "HashingVectorizer_50", "HashingVectorizer_1024"],
                       "acc" : [acc_unigram_tagger, acc_bigram_tagger, acc_trigram_tagger, acc_uni_bigramtagger, acc_uni_bi_trigramtagger, acc_HashingVectorizer, acc_HashingVectorizer_1024]})
result

,method,acc
0,UnigramTagger,0.823732
1,BigramTagger,0.829279
2,TrigramTagger,0.829143
3,Uni BigramTagger,0.827534
4,Uni Bi TrigramTagger,0.827905
5,HashingVectorizer_50,0.543369
6,HashingVectorizer_1024,0.885696


# Задание 2

Проверить, насколько хорошо работает NER
Данные брать из Index of /pub/named_entities
1. проверить NER из nltk/spacy/deeppavlov.
2. написать свой NER, попробовать разные подходы.
> - передаём в сетку токен и его соседей.
> - передаём в сетку только токен.
> - свой вариант.
3. сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.

In [1]:
!wget collection5.zip http://www.labinform.ru/pub/named_entities/collection5.zip

--2024-02-19 15:11:54--  http://collection5.zip/
Resolving collection5.zip (collection5.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘collection5.zip’
--2024-02-19 15:11:54--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.54MB/s    in 1.2s    

2024-02-19 15:11:56 (1.54 MB/s) - ‘collection5.zip’ saved [1899530/1899530]

FINISHED --2024-02-19 15:11:56--
Total wall clock time: 2.1s
Downloaded: 1 files, 1.8M in 1.2s (1.54 MB/s)


In [2]:
import shutil
import sys

from shutil import unpack_archive

In [3]:
unpack_archive('collection5.zip')

In [4]:
with open('Collection5/061.txt', encoding='utf-8') as file:
    text = file.read()

print(text)

Д.Медведев освободил от должности заместителя директора ФСИН

Президент России Дмитрий Медведев своим указом от 26 декабря освободил генерал-лейтенанта внутренней службы Александра Пискунова от должности заместителя директора Федеральной службы исполнения наказаний (ФСИН).

Этим же указом полковник внутренней службы Андрей Кошкин назначен начальником главного управления ФСИН по Нижегородской области. Ранее Д.Медведев назначил полковника внутренней службы Сергея Есипова начальником управления исполнения приговоров и специального учета ФСИН. Полковник внутренней службы Геннадий Наржонков был назначен начальником управления кадров ФСИН.

Полковник внутренней службы Владислав Цатуров вместо Валерия Телюхи занял пост начальника управления организации деятельности тюрем и следственных изоляторов ФСИН. Последний был отстранен от должности 11 декабря 2009г. в числе других высокопоставленных сотрудников службы.

Отметим, что отставкам крупных чиновников ФСИН предшествовало заявление министра юс

## NLTK

In [61]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [62]:
nltk.pos_tag(nltk.word_tokenize(text))

[('Д.Медведев', 'JJ'),
 ('освободил', 'NNP'),
 ('от', 'NNP'),
 ('должности', 'NNP'),
 ('заместителя', 'NNP'),
 ('директора', 'NNP'),
 ('ФСИН', 'NNP'),
 ('Президент', 'NNP'),
 ('России', 'NNP'),
 ('Дмитрий', 'NNP'),
 ('Медведев', 'NNP'),
 ('своим', 'NNP'),
 ('указом', 'NNP'),
 ('от', 'VBD'),
 ('26', 'CD'),
 ('декабря', 'JJ'),
 ('освободил', 'JJ'),
 ('генерал-лейтенанта', 'JJ'),
 ('внутренней', 'NN'),
 ('службы', 'NNP'),
 ('Александра', 'NNP'),
 ('Пискунова', 'NNP'),
 ('от', 'NNP'),
 ('должности', 'NNP'),
 ('заместителя', 'NNP'),
 ('директора', 'NNP'),
 ('Федеральной', 'NNP'),
 ('службы', 'NNP'),
 ('исполнения', 'NNP'),
 ('наказаний', 'NNP'),
 ('(', '('),
 ('ФСИН', 'NNP'),
 (')', ')'),
 ('.', '.'),
 ('Этим', 'JJ'),
 ('же', 'JJ'),
 ('указом', 'NN'),
 ('полковник', 'NNP'),
 ('внутренней', 'NNP'),
 ('службы', 'NNP'),
 ('Андрей', 'NNP'),
 ('Кошкин', 'NNP'),
 ('назначен', 'NNP'),
 ('начальником', 'NNP'),
 ('главного', 'NNP'),
 ('управления', 'NNP'),
 ('ФСИН', 'NNP'),
 ('по', 'NNP'),
 ('Нижего

In [68]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [69]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label') }

{('Андрей Кошкин', 'PERSON'),
 ('Валерия Телюхи', 'PERSON'),
 ('Отметим', 'PERSON'),
 ('России Дмитрий Медведев', 'PERSON'),
 ('Сергея Есипова', 'PERSON'),
 ('Сергея Магнитского', 'PERSON'),
 ('ФСИН', 'ORGANIZATION'),
 ('Цатуров', 'PERSON')}

## Spacy

In [73]:
import spacy
from spacy import displacy

In [75]:
!python -m spacy download ru_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 54.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [76]:
nlp = spacy.load("ru_core_news_md")

In [77]:
article = nlp(text)
displacy.render(article, jupyter=True, style='ent')

## Deeppavlov

In [5]:
!pip install deeppavlov

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.4/489.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Su

In [6]:
import deeppavlov
from deeppavlov import configs, build_model

In [7]:
!python -m deeppavlov install ner_ontonotes_bert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 75.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 

In [8]:
ner_model = build_model('ner_ontonotes_bert', download=True, install=True)

2024-02-19 15:15:46.437 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_torch_crf.tar.gz to /root/.deeppavlov/models/ner_ontonotes_bert_torch_crf.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_torch_crf.tar.gz to /root/.deeppavlov/models/ner_ontonotes_bert_torch_crf.tar.gz
100%|██████████| 1.13G/1.13G [00:57<00:00, 19.8MB/s]
2024-02-19 15:16:44.751 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting /root/.deeppavlov/models/ner_ontonotes_bert_torch_crf.tar.gz archive into /root/.deeppavlov/models/ner_ontonotes_bert_torch_crf
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/models/ner_ontonotes_bert_torch_crf.tar.gz archive into /root/.deeppavlov/models/ner_ontonotes_bert_torch_crf
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Col

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['cl

In [10]:
ner_model([text[:512]])

/usr/local/lib/python3.10/dist-packages/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:413.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


[[['Д.Медведев',
   'освободил',
   'от',
   'должности',
   'заместителя',
   'директора',
   'ФСИН',
   '\n',
   '\n',
   'Президент',
   'России',
   'Дмитрий',
   'Медведев',
   'своим',
   'указом',
   'от',
   '26',
   'декабря',
   'освободил',
   'генерал',
   '-',
   'лейтенанта',
   'внутренней',
   'службы',
   'Александра',
   'Пискунова',
   'от',
   'должности',
   'заместителя',
   'директора',
   'Федеральной',
   'службы',
   'исполнения',
   'наказаний',
   '(',
   'ФСИН',
   ')',
   '.',
   '\n',
   '\n',
   'Этим',
   'же',
   'указом',
   'полковник',
   'внутренней',
   'службы',
   'Андрей',
   'Кошкин',
   'назначен',
   'начальником',
   'главного',
   'управления',
   'ФСИН',
   'по',
   'Нижегородской',
   'области.',
   'Ранее',
   'Д.Медведев',
   'назначил',
   'полковника',
   'внутренней',
   'службы',
   'Сергея',
   'Есипова',
   'начальником',
   'управления',
   'исполнения',
   'приг']],
 [['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O'

In [11]:
ner_model_ner_collection3_bert = build_model('ner_collection3_bert', download=True, install=True)

2024-02-19 15:20:54.62 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/v1/ner/ner_rus_bert_coll3_torch.tar.gz to /root/.deeppavlov/models/ner_rus_bert_coll3_torch.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/v1/ner/ner_rus_bert_coll3_torch.tar.gz to /root/.deeppavlov/models/ner_rus_bert_coll3_torch.tar.gz
100%|██████████| 1.44G/1.44G [01:16<00:00, 18.8MB/s]
2024-02-19 15:22:11.412 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting /root/.deeppavlov/models/ner_rus_bert_coll3_torch.tar.gz archive into /root/.deeppavlov/models/ner_rus_bert_coll3_torch
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/models/ner_rus_bert_coll3_torch.tar.gz archive into /root/.deeppavlov/models/ner_rus_bert_coll3_torch


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initializ

In [12]:
ner_model_ner_collection3_bert([text[:512]])

[[['Д.Медведев',
   'освободил',
   'от',
   'должности',
   'заместителя',
   'директора',
   'ФСИН',
   '\n',
   '\n',
   'Президент',
   'России',
   'Дмитрий',
   'Медведев',
   'своим',
   'указом',
   'от',
   '26',
   'декабря',
   'освободил',
   'генерал',
   '-',
   'лейтенанта',
   'внутренней',
   'службы',
   'Александра',
   'Пискунова',
   'от',
   'должности',
   'заместителя',
   'директора',
   'Федеральной',
   'службы',
   'исполнения',
   'наказаний',
   '(',
   'ФСИН',
   ')',
   '.',
   '\n',
   '\n',
   'Этим',
   'же',
   'указом',
   'полковник',
   'внутренней',
   'службы',
   'Андрей',
   'Кошкин',
   'назначен',
   'начальником',
   'главного',
   'управления',
   'ФСИН',
   'по',
   'Нижегородской',
   'области.',
   'Ранее',
   'Д.Медведев',
   'назначил',
   'полковника',
   'внутренней',
   'службы',
   'Сергея',
   'Есипова',
   'начальником',
   'управления',
   'исполнения',
   'приг']],
 [['S-PER',
   'O',
   'O',
   'O',
   'O',
   'O',
   'S-ORG'

In [16]:
ner_model_ner_collection3_bert([text[:95]])

[[['Д.Медведев',
   'освободил',
   'от',
   'должности',
   'заместителя',
   'директора',
   'ФСИН',
   '\n',
   '\n',
   'Президент',
   'России',
   'Дмитрий',
   'Медведев']],
 [['S-PER',
   'O',
   'O',
   'O',
   'O',
   'O',
   'S-ORG',
   'O',
   'O',
   'O',
   'S-LOC',
   'B-PER',
   'E-PER']]]

## Natasha

In [17]:
!pip install natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 83.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=56efb58d0dca940425e753e0ad4aab654a128e3656686c848815351d092d9237
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26095 sha256=60689fc7222b86f9ada42bd8c1b91f8f757ede77e60f3a2081ad2c6d75f6673f
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built docopt intervaltree


In [18]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)

In [21]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [19]:
doc = Doc(text)

In [22]:
doc.segment(segmenter)
display(doc.tokens[:10])
display(doc.sents[:10])

[DocToken(stop=1, text='Д'),
 DocToken(start=1, stop=2, text='.'),
 DocToken(start=2, stop=10, text='Медведев'),
 DocToken(start=11, stop=20, text='освободил'),
 DocToken(start=21, stop=23, text='от'),
 DocToken(start=24, stop=33, text='должности'),
 DocToken(start=34, stop=45, text='заместителя'),
 DocToken(start=46, stop=55, text='директора'),
 DocToken(start=56, stop=60, text='ФСИН'),
 DocToken(start=62, stop=71, text='Президент')]

[DocSent(stop=273, text='Д.Медведев освободил от должности заместителя дир..., tokens=[...]),
 DocSent(start=275, stop=403, text='Этим же указом полковник внутренней службы Андрей..., tokens=[...]),
 DocSent(start=404, stop=545, text='Ранее Д.Медведев назначил полковника внутренней с..., tokens=[...]),
 DocSent(start=546, stop=641, text='Полковник внутренней службы Геннадий Наржонков бы..., tokens=[...]),
 DocSent(start=643, stop=806, text='Полковник внутренней службы Владислав Цатуров вме..., tokens=[...]),
 DocSent(start=807, stop=915, text='Последний был отстранен от должности 11 декабря 2..., tokens=[...]),
 DocSent(start=917, stop=1186, text='Отметим, что отставкам крупных чиновников ФСИН пр..., tokens=[...]),
 DocSent(start=1187, stop=1260, text='По предварительным данным, смерть наступила от се..., tokens=[...]),
 DocSent(start=1262, stop=1342, text='Юрист проходил обвиняемым по делу об уклонении от..., tokens=[...]),
 DocSent(start=1343, stop=1614, text='С.Магнитскому было пред

In [23]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
sent = doc.sents[0]
sent.morph.print()

                   Д PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
                   . PUNCT
            Медведев PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
           освободил VERB|Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
                  от ADP
           должности NOUN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
         заместителя NOUN|Animacy=Anim|Case=Acc|Gender=Masc|Number=Sing
           директора NOUN|Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing
                ФСИН PROPN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
           Президент PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
              России PROPN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
             Дмитрий PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
            Медведев PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
               своим DET|Case=Ins|Gender=Masc|Number=Sing
              указом NOUN|Animacy=Inan|Case=Ins|Gender=Masc|N

In [24]:
sent.syntax.print()

                  ┌► Д                  nsubj
                  │  .                  
                  │  Медведев           
┌───┌───┌─┌─┌─┌─┌─└─ освободил          
│   │   │ │ │ │ │ ┌► от                 case
│   │   │ │ │ │ └►└─ должности          obl
│   │ ┌─│ │ │ └──►┌─ заместителя        obj
│   │ │ │ │ │     └► директора          nmod
│ ┌►│ │ │ │ │        ФСИН               nmod
│ │ │ │ │ │ └────►┌─ Президент          nsubj
│ │ │ │ │ │       └► России             nmod
│ │ │ │ │ │ ┌────►┌─ Дмитрий            appos
│ │ │ │ │ │ │     └► Медведев           flat:name
│ │ │ │ │ │ │     ┌► своим              det
│ │ │ │ │ └►│     └─ указом             obl
│ │ │ │ │   │     ┌► от                 case
│ │ │ │ │   │ ┌►┌─└─ 26                 obl
│ │ │ │ │   │ │ └──► декабря            flat
│ │ │ │ │   │ └───── освободил          
│ │ │ │ └►┌─└─┌─┌─── генерал-лейтенанта obj
│ │ │ │   │   │ │ ┌► внутренней         amod
│ │ │ │   │   │ └►└─ службы             nmod
│ │ │ │   │   └──►┌─ Але

In [25]:
doc.tag_ner(ner_tagger)
doc.ner.print()

Д.Медведев освободил от должности заместителя директора ФСИН
                                                        ORG─
Президент России Дмитрий Медведев своим указом от 26 декабря освободил
          LOC─── PER─────────────                                     
 генерал-лейтенанта внутренней службы Александра Пискунова от 
                                      PER─────────────────    
должности заместителя директора Федеральной службы исполнения 
                                ORG───────────────────────────
наказаний (ФСИН).
──────────────── 
Этим же указом полковник внутренней службы Андрей Кошкин назначен 
                                           PER──────────          
начальником главного управления ФСИН по Нижегородской области. Ранее 
                                ORG─    LOC──────────────────        
Д.Медведев назначил полковника внутренней службы Сергея Есипова 
PER───────                                       PER─────────── 
начальником управления исполнения приговоров